In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import r2_score

In [2]:
def split_data(df, df_test, atributos):
    X = df[atributos]
    y = df.SalePrice
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_pred = test[atributos]
    X_pred = scaler.fit_transform(X_pred)
    return X_train, X_test, y_train, y_test, X_pred

def ret_df(pred, nombre):
    df_reg = pd.DataFrame()
    df_reg['Id'] = test.Id
    df_reg['SalePrice'] = pd.Series(pred)
    df_reg.set_index('Id', inplace = True)
    df_reg.to_csv("D:/DatosD/Ml/Supervizado/Kaggle/House_price/Resultados/" + str(nombre) + ".csv")

In [3]:
df = pd.read_csv('Train_data.csv')
test = pd.read_csv('Test_data.csv')

In [4]:
atrib = ['LotFrontage', 'LotArea', 'YearBuilt', 'CentralAir', 'GrLivArea',
       'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageArea',
       'OpenPorchSF', 'Condition', 'Mean_rank', 'YearRemod',
       'Exterior', 'TotalSF', 'TotalBath', 'WoodYN', 'Age']
#atributos = ['TotalSF', 'GrLivArea', 'TotalBath', 'GarageArea', 'Mean_rank', 'Age']


X_train, X_test, y_train, y_test, X_pred = split_data(df, test, atrib)

- booster: El tipo de modelo de clasificación usado, por defecto gbtree.
- objective: El tipo de tarea de clasificación que realizaremos. Para clasificación binaria, nuestro caso, especificamos binary:logistic.
- max_depth: “Profundidad” o número de nodos de bifurcación de los árboles de decisión usados en el entrenamiento. Aunque una mayor profundidad puede devolver mejores resultados, también puede resultar en overfitting (sobre ajuste).
- eta: La tasa de aprendizaje del modelo. Un mayor valor llega más rápidamente al mínimo de la función objetivo, es decir, a un “mejor modelo”, pero puede “pasarse” de su valor óptimo. En cambio, un valor pequeño puede nunca llegar al valor óptimo de la función objetivo, incluso después de muchas iteraciones. En ambos casos, esto afecta el desempeño de nuestros modelos con nuevos.
- nthread: El número de hilos computacionales que serán usados en el proceso de entrenamiento. Generalmente se refiere a los núcleos del procesador de tu equipo de cómputo, local o remoto, pero también pueden ser los núcleos de un GPU.
- nround: El número de iteraciones que se realizarán antes de detener el proceso de ajuste. Un mayor número de iteraciones generalmente devuelve mejores resultados de predicción, pero necesita más tiempo de entrenamiento y conlleva un riesgo de sobre ajuste si son demasiadas rondas.

In [5]:
model = xgb.XGBRegressor()

In [6]:
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [7]:
y_pred = model.predict(X_test)
r2_score(y_test, y_pred)

0.8800505473179986

In [8]:
atributos = ['TotalSF', 'GrLivArea', 'TotalBath', 'GarageArea', 'Mean_rank', 'Age']

X_train, X_test, y_train, y_test, X_pred = split_data(df, test, atributos)

In [9]:
modelo = xgb.XGBRegressor()
modelo.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [10]:
y_pred = modelo.predict(X_test)
r2_score(y_test, y_pred)

0.8754189053674506

In [11]:
atributos = ['TotalSF', 'GrLivArea', 'TotalBath', 'Mean_rank', 'YearBuilt']
X_train, X_test, y_train, y_test, X_pred = split_data(df, test, atributos)

In [12]:
modelo = xgb.XGBRegressor()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)
r2_score(y_test, y_pred)

0.8910791814215876

In [13]:
y_pred = modelo.predict(X_pred)
ret_df(y_pred, 'G_Xgboost')

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

In [17]:
param_grid = {
    'n_estimators': [100, 500, 900, 1000],
    'learning_rate': [0.05,0.1],
    'max_depth': [2, 10, 15],
    'min_child_weight': [1,4]
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print(grid_search.best_params_)

{'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 4, 'n_estimators': 100}


In [23]:
rdo = grid_search.best_estimator_
y_pred = rdo.predict(X_test)

In [24]:
r2_score(y_test, y_pred)

0.8537613956421088

In [25]:
fin = rdo.predict(X_pred)
ret_df(fin, 'H_XGB_boost')